In [6]:
# Load filter2D + dilate overlay
from pynq import Overlay
bareHDMI = Overlay("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.bit")
bareHDMI.download()
import pynq_cv.overlays.xv2Filter2DDilate as xv2

# Load xlnk memory mangager
from pynq import Xlnk
Xlnk.set_allocator_library("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.so")
mem_manager = Xlnk()

hdmi_in = bareHDMI.video.hdmi_in
hdmi_out = bareHDMI.video.hdmi_out

In [7]:
from pynq.lib.video import *
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.cacheable_frames = False
hdmi_out.cacheable_frames = False

hdmi_in.start()
hdmi_out.start()

In [8]:
mymode = hdmi_in.mode
print("My mode: "+str(mymode))

height = hdmi_in.mode.height
width = hdmi_in.mode.width
bpp = hdmi_in.mode.bits_per_pixel

My mode: VideoMode: width=1280 height=720 bpp=8


In [9]:
hdmi_in.tie(hdmi_out)

In [10]:
import time

numframes = 60
start = time.time()

for _ in range(numframes):
    f = hdmi_in.readframe()
    hdmi_out.writeframe(f)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Frames per second:  60.60070392281893


In [ ]:
# import PIL.Image
# image = PIL.Image.fromarray(hdmi_in.readframe())
# image

In [ ]:
# from pynq import Overlay
# bs = Overlay("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.bit")
# bs.download()

In [11]:
# from pynq import Xlnk
# Xlnk.set_allocator_library('/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.so')
# mem_manager = Xlnk()

# import pynq_cv.overlays.xv2Filter2DDilate as xv2
import numpy as np
import cv2
import time


dstSW = np.ones((height,width),np.uint8);

# xFimgY  = mem_manager.cma_array((height,width),np.uint8) #allocated physically contiguous numpy array 
# xFimgY[:] = imgY[:] # copy source data

xFdst  = mem_manager.cma_array((height,width),np.uint8) #allocated physically contiguous numpy array

In [37]:
kernel = {
        'average blur': np.array([
            [1.0, 1.0, 1.0],
            [1.0, 1.0, 1.0],
            [1.0, 1.0, 1.0]],np.float32)/9,
        'gaussian blur': np.array([
            [1.0, 2.0, 1.0],
            [2.0, 4.0, 2.0],
            [1.0, 2.0, 1.0]],np.float32)/16,
        'sobel ver': np.array([
            [1.0,0.0,-1.0],
            [2.0,0.0,-2.0],
            [1.0,0.0,-1.0]],np.float32),
        'sobel hor': np.array([
            [1.0,2.0,1.0],
            [0.0,0.0,0.0],
            [-1.0,-2.0,-1.0]],np.float32),
        'laplacian': np.array([
            [0.0, 1.0, 0],
            [1.0, -4, 1.0],
            [0, 1.0, 0.0]],np.float32),
        'sharpen': np.array([
            [-1,-1, -1],
            [-1, 9, -1],
            [-1, -1, -1]],np.float32),
}

numberOfIterations=200
kernel_name = 'laplacian'

In [ ]:
# kernel.get(kernel_name)

## Software

In [42]:
# kernel_name = 'laplacian'
startSW=time.time()
for i in range(numberOfIterations):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    cv2.filter2D(inframe, -1, kernel.get(kernel_name), dst=outframe, borderType=cv2.BORDER_CONSTANT) #filter2D on ARM
    hdmi_out.writeframe(outframe)
stopSW=time.time()
print("Start SW loop = ", (stopSW - startSW))
print("SW frames per second: ", ((numberOfIterations) / (stopSW - startSW)))

Start SW loop =  24.78199815750122
SW frames per second:  8.070374258318727


## Hardware

In [ ]:
# kernel_name = 'laplacian'
startPL=time.time()
for i in range(numberOfIterations):
    inframe = hdmi_in.readPYNQ-Skripsi/frame()
    outframe = hdmi_out.newframe()
    xv2.filter2D(inframe, -1, kernel.get(kernel_name), dst=outframe,borderType=cv2.BORDER_CONSTANT) #filter2D offloaded to PL, working on physically continuous numpy arrays
    hdmi_out.writeframe(outframe)
stopPL=time.time()
print("Start HW loop = ", (stopPL - startPL))
print("PL frames per second: ", ((numberOfIterations) / (stopPL - startPL)))

In [ ]:
outframe

In [ ]:
hdmi_out.close()
hdmi_in.close()